In [1]:
def create_slice(index_names,*vargs,**kwargs):
    return tuple(map(lambda elm:f(elm[0],elm[1],vargs,kwargs) ,enumerate(index_names)))

In [2]:
def f(i,indx,vargs,kwargs):
    if i<len(vargs):
        return vargs[i]
    else:
        return kwargs.get(indx,slice(None))
    

In [3]:
index_names = ["index_1","index_2"]

In [4]:
index_names

['index_1', 'index_2']

In [5]:
create_slice(index_names,3)

(3, slice(None, None, None))

In [6]:
import pandas as pd

In [7]:
# @pd.api.extensions.register_dataframe_accessor("fidx")
class FastIndex(object):
    def __init__(self, df):
        self._df = df

    @staticmethod
    def _validate(obj):
        pass


    def f_slice(self,*vargs,**kwargs):
        # return the geographic center point of this DataFrame
        slc = create_slice(self._df.index.names,*vargs,**kwargs)
        if isinstance(self._df.index, pd.core.index.MultiIndex):    
            return slc
        else:
            return slc[0]
    
    def slice(self,*vargs,columns =slice(None),**kwargs):
        return self._df.loc[self.f_slice(*vargs,**kwargs),columns]
  


  

In [8]:
pd.api.extensions.register_dataframe_accessor("fidx")(FastIndex)


__main__.FastIndex

In [9]:
pd.MultiIndex.from_tuples([],names=["indx1","indx2"])

MultiIndex(levels=[[], []],
           codes=[[], []],
           names=['indx1', 'indx2'])

In [10]:
df = pd.DataFrame([[1,2,3,4],[3,4,5,6]],index=pd.MultiIndex.from_tuples([("one","a"),("two","b")],names=["indx1","indx2"]))

In [11]:
df1 = pd.DataFrame([[1,2,3,4],[3,4,5,6]])

In [12]:
df1.loc[[0]]

,0,1,2,3
0,1,2,3,4


In [13]:
df1.loc[df1.fidx.f_slice([0])[0],slice(None)]

0    1
1    2
2    3
3    4
Name: 0, dtype: int64

In [14]:
df1.fidx.slice([1])

,0,1,2,3
1,3,4,5,6


In [15]:
df1.loc[df1.fidx.f_slice([0]),slice(None)]

,0,1,2,3
0,1,2,3,4


In [16]:
df.fidx.f_slice(indx="df")

(slice(None, None, None), slice(None, None, None))

In [17]:
df1.loc[slice(0)]

,0,1,2,3
0,1,2,3,4


In [28]:
df.fidx.slice(indx2=pd.IndexSlice["b"],columns=[0])

,,0
indx1,indx2,
two,b,3


In [29]:
df.loc[("one","a"),:]

0    1
1    2
2    3
3    4
Name: (one, a), dtype: int64

In [30]:
path = "./example_data/fifa19.csv"

In [32]:
df = pd.read_csv(path)

In [33]:
df.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [35]:
df.set_index(["Nationality","Age","Club","ID"],inplace=True)

In [39]:
df.sort_index(inplace=True)

In [ ]:
df.index.set_names(["nationality","age","club","id"],inplace=True)

lets say we want the all the spanish players
we could do:

In [50]:
df.loc["Spain"]

Unnamed: 0             Name  \
age club                    id                                    
16  CF Rayo Majadahonda     246465       12828         Guerrero   
    Real Sporting de Gijón  245616       12496   Pelayo Morilla   
17  Atlético Madrid         245999       15786          Mollejo   
    Borussia Dortmund       241811        7217     Sergio Gómez   
    Elche CF                243943       16224       Javi López   
    Manchester City         245037       13211      Eric García   
    Málaga CF               246070       15422       Iván Jaime   
    RCD Mallorca            246455       15104      Pablo Ramón   
    Rayo Vallecano          245632       16467            Morro   
                            246278       14662     Manu Navarro   
    Real Betis              245894       14321           Robert   
    SD Huesca               246593       16756           Javito   
18  Albacete BP             243156       15168    Álvaro García   
                            245599       16792      Darío Ramos   
    Athletic Club de Bilbao 239614       12059      Jesús Areso   
                            244675        8502           Sancet   
    Atlético Madrid         240559       15645            Obama   
                            243652       11805     Borja Garcés   
    CF Reus Deportiu        245705       15697      Iker Goujón   
    Córdoba CF              246044       13157    Andrés Martín   
    FC Barcelona            237522       10777     Jorge Cuenca   
                            239250        8289        Abel Ruiz   
                            240915        8857          Miranda   
                            242816        6102       Riqui Puig   
    Getafe CF               243032       13367        Hugo Duro   
    Girona FC               243576        6076      Pedro Porro   
                            246351       15967           Valery   
    Manchester City         231410        6559      Brahim Díaz   
    RC Celta                241055       14393     Diego Pampín   
    RCD Espanyol            244407       16556       Pol Lozano   
...                                        ...              ...   
35  AD Alcorcón             145117        4627            Dorca   
    CD Lugo                 145996        2224           Seoane   
    CF Reus Deportiu        171565        5807          Linares   
    Extremadura UD          171017        8805      Íñigo López   
    Milan                   24630          224       Pepe Reina   
    Perth Glory             147006        1692     Diego Castro   
    Queens Park Rangers     153938        3464     Àngel Rangel   
    RCD Espanyol            135697         911    Sergio García   
    Rayo Vallecano          110019        4551           Dorado   
    Real Oviedo             157270        5941            Toché   
    UD Las Palmas           146058        5238             Momo   
36  Getafe CF               154472        1019     Jorge Molina   
    New York City FC        113422         315      David Villa   
    RCD Espanyol            146748         544      Diego López   
    RCD Mallorca            110680        9179     Xisco Campos   
    Rayo Vallecano          139597        5039      Javi Guerra   
    Real Betis              45186          411          Joaquín   
    UD Las Palmas           146059        3644     David García   
    Venezia FC              135743        5856       Alex Geijo   
37  Athletic Club de Bilbao 106231         316           Aduriz   
    Brighton & Hove Albion  145047        2380     Bruno Saltor   
    FC Porto                5479           318         Casillas   
    Girona FC               114764        1720          Iraizoz   
    KAS Eupen               16            4014      Luis García   
    Real Valladolid CF      110787        5434  Borja Fernández   
    SD Huesca               52842         8122          Camacho   
    UD Las Palmas           115909        1137     Rubén Castro   
38  Elche CF    

to get all the spanish players who are 25 we can do

In [54]:
df.loc["Spain",25]

Unnamed: 0               Name  \
club                    id                                      
AD Alcorcón             204783       11990             Felipe   
                        233470        9920        Borja Galán   
                        233588        8157          Elgezabal   
Albacete BP             199491       10208        Fran García   
                        214930        6175               Caro   
Athletic Club de Bilbao 189575         501            Muniain   
                        225523        2046              Lekue   
Barnsley                211209        5677           Pinillos   
Bournemouth             220414        1568         Diego Rico   
CA Osasuna              215569        6675        Unai García   
                        224003        3975     Sergio Herrera   
CD Leganés              146947        3218              Vesga   
CD Lugo                 201401        7355         Campabadal   
                        229655       11092       Alberto Varo   
CD Numancia             205158        4282          Guillermo   
                        210110        8473              Derik   
                        211056        7067      Luis Valcarce   
                        224981       10791       Grego Sierra   
                        228780       11699          Kako Sanz   
                        230849        8489              Nacho   
CD Tenerife             215572        5715            Alberto   
                        217277       10688       Ángel Galván   
CF Rayo Majadahonda     220348        9619     Isaac Carcelén   
CF Reus Deportiu        235754        5574      Borja Herrera   
Chelsea                 201153         265             Morata   
Cádiz CF                199247        3454     Álex Fernández   
                        208638        9917       Rober Correa   
                        216568        7606        Mario Barco   
Deportivo Alavés        194996        1940       Borja Bastón   
Deportivo de La Coruña  210930        2044         Carles Gil   
...                                    ...                ...   
Girona FC               211706         799          Pere Pons   
Granada CF              234781       11121           Martínez   
Lazio                   198706         271       Luis Alberto   
                        206098        3273             Patric   
Levante UD              199823         640            Campaña   
Liverpool               205566        1194     Alberto Moreno   
Orlando City SC         204677        6123       Oriol Rosell   
Paris Saint-Germain     202515        1223               Jesé   
                        205069        1218             Bernat   
Pogoń Szczecin          189573        8230        Guarrotxena   
RC Celta                228716        7061              Kevin   
RCD Espanyol            224179        1103     Borja Iglesias   
RCD Mallorca            202716        9754              Abdón   
                        211057        6049     Pablo Valcarce   
Rayo Vallecano          216447        2329      Álvaro García   
                        217036        2505        Álex Moreno   
                        220355        4549       Alex Alegría   
Real Sociedad           197891        1201             Juanmi   
                        204523        2031        Rubén Pardo   
Real Sporting de Gijón  219840       10702           Noblejas   
                        230793       10490      Álvaro Traver   
Real Zaragoza           234994        6560              Ratón   
SD Eibar                204526        3168            Hervías   
                        225926        2928         Pere Milla   
Sandefjord Fotball      220396        8685               Rufo   
Sint-Truidense VV       205353        6772  Cristian Ceballos   
Sparta Praha            240035        8366     Néstor Albiach   
UD Almería              219837        5410              Aguza   
UD Las Palmas           205551        4691       Álvaro Lemos   
Wi

but what if we like all the players which are 25 years old from all nationalities?
you would try:

In [55]:
df.loc[:,25]

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [23] of <class 'int'>

or

In [57]:
df.loc[(:,25),:]

SyntaxError: invalid syntax (<ipython-input-57-81e43cf84b25>, line 1)

but this results in an error

the correct way to do it is

In [59]:
df.loc[(slice(None),25),:]

Unnamed: 0               Name  \
nationality age club                   id                                      
Albania     25  Atalanta               208165        7815        B. Djimsiti   
                Empoli                 203780        4767          F. Veseli   
                Neuchâtel Xamax        244581       14912      A. Fejzullahu   
Algeria     25  Dijon FCO              188600        3770           M. Abeid   
                FC Metz                210438        3232         F. Boulaya   
                FC Sion                243896       15665      A. Abdellaoui   
                Le Havre AC            222515        3371          Z. Ferhat   
Angola      25  Club Brugge KV         208364        3902       Clinton Mata   
                Livingston FC          207651       12207        Dolly Menga   
                Zagłębie Sosnowiec     209842       14447         Christovao   
Argentina   25  Ajax                   211256         445      N. Tagliafico   
                Argentinos Juniors     214776        5612         O. Benítez   
                                       226675       11460     F. Lanzillotta   
                Arsenal                202811        3392        E. Martínez   
                Atalanta               215259         784          E. Rigoni   
                Atlético Tucumán       227602        5706         F. Álvarez   
                Bayer 04 Leverkusen    215353         591          L. Alario   
                Belgrano de Córdoba    215328        6728      G. Gil Romero   
                CD Antofagasta         219518        9330          P. Becker   
                Club Atlético Banfield 222018       11280         R. Arciero   
                Club Atlético Colón    215070        2119           G. Ortíz   
                                       223593        3536          J. Correa   
                Club Atlético Huracán  215168        3615           J. Garro   
                                       219931        7766         D. Mendoza   
                                       242080       12693          G. Guerra   
                Club Atlético Lanús    202331        3492         F. Quignon   
                                       214779        8752           B. Vides   
                Club Atlético Talleres 214665        2931         J. Ramírez   
                Club Deportes Temuco   227876       12214         L. Campana   
                Clube Sport Marítimo   220938        3245          J. Correa   
...                                                   ...                ...   
Uruguay     25  Independiente Santa Fe 205972       12643     R. Bentancourt   
                Jaguares de Córdoba    224078       15471          F. Pintos   
                KV Kortrijk            219988        4236        F. Avenatti   
                Milan                  213400         790          D. Laxalt   
                Millonarios FC         239645        5749   M. de los Santos   
                Palermo                220084        3759        C. Falletti   
                Patronato              235950       13663         F. Barceló   
                Pescara                205726        3698         G. Brugman   
                San Martín de San Juan 244954       13813   A. Silva Quiroga   
                Sivasspor              223690        6390          S. Rochet   
                Unión Española         239104        7648        A. Pizzorno   
                Unión La Calera        227838        6606           C. Núñez   
                                       242209        7797       A. Rodriguez   
                Vancouver Whitecaps FC 235280        7715             J. Aja   
Venezuela   25  Al Wehda               217097        1590           R. Otero   
                Atlanta United         207877         345        J. Martínez   
                CF Reus Deportiu       233329        7076      M. Villanueva   
                Elche CF               207876  

or 

In [60]:
df.query("age==25") #this returns a copy which means you cant assing values this way...

Unnamed: 0               Name  \
nationality age club                   id                                      
Albania     25  Atalanta               208165        7815        B. Djimsiti   
                Empoli                 203780        4767          F. Veseli   
                Neuchâtel Xamax        244581       14912      A. Fejzullahu   
Algeria     25  Dijon FCO              188600        3770           M. Abeid   
                FC Metz                210438        3232         F. Boulaya   
                FC Sion                243896       15665      A. Abdellaoui   
                Le Havre AC            222515        3371          Z. Ferhat   
Angola      25  Club Brugge KV         208364        3902       Clinton Mata   
                Livingston FC          207651       12207        Dolly Menga   
                Zagłębie Sosnowiec     209842       14447         Christovao   
Argentina   25  Ajax                   211256         445      N. Tagliafico   
                Argentinos Juniors     214776        5612         O. Benítez   
                                       226675       11460     F. Lanzillotta   
                Arsenal                202811        3392        E. Martínez   
                Atalanta               215259         784          E. Rigoni   
                Atlético Tucumán       227602        5706         F. Álvarez   
                Bayer 04 Leverkusen    215353         591          L. Alario   
                Belgrano de Córdoba    215328        6728      G. Gil Romero   
                CD Antofagasta         219518        9330          P. Becker   
                Club Atlético Banfield 222018       11280         R. Arciero   
                Club Atlético Colón    215070        2119           G. Ortíz   
                                       223593        3536          J. Correa   
                Club Atlético Huracán  215168        3615           J. Garro   
                                       219931        7766         D. Mendoza   
                                       242080       12693          G. Guerra   
                Club Atlético Lanús    202331        3492         F. Quignon   
                                       214779        8752           B. Vides   
                Club Atlético Talleres 214665        2931         J. Ramírez   
                Club Deportes Temuco   227876       12214         L. Campana   
                Clube Sport Marítimo   220938        3245          J. Correa   
...                                                   ...                ...   
Uruguay     25  Independiente Santa Fe 205972       12643     R. Bentancourt   
                Jaguares de Córdoba    224078       15471          F. Pintos   
                KV Kortrijk            219988        4236        F. Avenatti   
                Milan                  213400         790          D. Laxalt   
                Millonarios FC         239645        5749   M. de los Santos   
                Palermo                220084        3759        C. Falletti   
                Patronato              235950       13663         F. Barceló   
                Pescara                205726        3698         G. Brugman   
                San Martín de San Juan 244954       13813   A. Silva Quiroga   
                Sivasspor              223690        6390          S. Rochet   
                Unión Española         239104        7648        A. Pizzorno   
                Unión La Calera        227838        6606           C. Núñez   
                                       242209        7797       A. Rodriguez   
                Vancouver Whitecaps FC 235280        7715             J. Aja   
Venezuela   25  Al Wehda               217097        1590           R. Otero   
                Atlanta United         207877         345        J. Martínez   
                CF Reus Deportiu       233329        7076      M. Villanueva   
                Elche CF               207876  

another example is selecting all the spanish players from Arsenal club
the pandas way of chosing would be
two ways you could do it is by query and loc:

In [65]:
df.query("nationality=='Spain' and club=='Arsenal'")

Unnamed: 0             Name  \
nationality age club    id                                    
Spain       21  Arsenal 222576        7069      Pleguezuelo   
            23  Arsenal 203747         464  Héctor Bellerín   
            32  Arsenal 177604         520    Nacho Monreal   

                                                                         Photo  \
nationality age club    id                                                       
Spain       21  Arsenal 222576  https://cdn.sofifa.org/players/4/19/222576.png   
            23  Arsenal 203747  https://cdn.sofifa.org/players/4/19/203747.png   
            32  Arsenal 177604  https://cdn.sofifa.org/players/4/19/177604.png   

                                                               Flag  Overall  \
nationality age club    id                                                     
Spain       21  Arsenal 222576  https://cdn.sofifa.org/flags/45.png       68   
            23  Arsenal 203747  https://cdn.sofifa.org/flags/45.png       80   
            32  Arsenal 177604  https://cdn.sofifa.org/flags/45.png       80   

                                Potential  \
nationality age club    id                  
Spain       21  Arsenal 222576         77   
            23  Arsenal 203747         85   
            32  Arsenal 177604         80   

                                                                 Club Logo  \
nationality age club    id                                                   
Spain       21  Arsenal 222576  https://cdn.sofifa.org/teams/2/light/1.png   
            23  Arsenal 203747  https://cdn.sofifa.org/teams/2/light/1.png   
            32  Arsenal 177604  https://cdn.sofifa.org/teams/2/light/1.png   

                                 Value  Wage  Special  ... Composure  Marking  \
nationality age club    id                             ...                      
Spain       21  Arsenal 222576   €1.2M  €21K     1672  ...      59.0     65.0   
            23  Arsenal 203747  €16.5M  €85K     1990  ...      70.0     75.0   
            32  Arsenal 177604     €8M  €99K     2011  ...      76.0     80.0   

                                StandingTackle  SlidingTackle GKDiving  \
nationality age club    id                                               
Spain       21  Arsenal 222576            68.0           70.0     14.0   
            23  Arsenal 203747            78.0           80.0      8.0   
            32  Arsenal 177604            83.0           81.0      7.0   

                               GKHandling GKKicking GKPositioning  GKReflexes  \
nationality age club    id                                                      
Spain       21  Arsenal 222576       15.0       8.0          11.0         9.0   
            23  Arsenal 203747       14.0      14.0          13.0        12.0   
            32  Arsenal 177604        5.0       9.0           3.0         5.0   

                               Release Clause  
nationality age club    id                     
Spain       21  Arsenal 222576          €2.5M  
            23  Arsenal 203747         €32.6M  
            32  Arsenal 177604         €15.2M  

[3 rows x 85 columns]

In [66]:
df.loc[("Spain",slice(None),"Arsenal"),:]

Unnamed: 0             Name  \
nationality age club    id                                    
Spain       21  Arsenal 222576        7069      Pleguezuelo   
            23  Arsenal 203747         464  Héctor Bellerín   
            32  Arsenal 177604         520    Nacho Monreal   

                                                                         Photo  \
nationality age club    id                                                       
Spain       21  Arsenal 222576  https://cdn.sofifa.org/players/4/19/222576.png   
            23  Arsenal 203747  https://cdn.sofifa.org/players/4/19/203747.png   
            32  Arsenal 177604  https://cdn.sofifa.org/players/4/19/177604.png   

                                                               Flag  Overall  \
nationality age club    id                                                     
Spain       21  Arsenal 222576  https://cdn.sofifa.org/flags/45.png       68   
            23  Arsenal 203747  https://cdn.sofifa.org/flags/45.png       80   
            32  Arsenal 177604  https://cdn.sofifa.org/flags/45.png       80   

                                Potential  \
nationality age club    id                  
Spain       21  Arsenal 222576         77   
            23  Arsenal 203747         85   
            32  Arsenal 177604         80   

                                                                 Club Logo  \
nationality age club    id                                                   
Spain       21  Arsenal 222576  https://cdn.sofifa.org/teams/2/light/1.png   
            23  Arsenal 203747  https://cdn.sofifa.org/teams/2/light/1.png   
            32  Arsenal 177604  https://cdn.sofifa.org/teams/2/light/1.png   

                                 Value  Wage  Special  ... Composure  Marking  \
nationality age club    id                             ...                      
Spain       21  Arsenal 222576   €1.2M  €21K     1672  ...      59.0     65.0   
            23  Arsenal 203747  €16.5M  €85K     1990  ...      70.0     75.0   
            32  Arsenal 177604     €8M  €99K     2011  ...      76.0     80.0   

                                StandingTackle  SlidingTackle GKDiving  \
nationality age club    id                                               
Spain       21  Arsenal 222576            68.0           70.0     14.0   
            23  Arsenal 203747            78.0           80.0      8.0   
            32  Arsenal 177604            83.0           81.0      7.0   

                               GKHandling GKKicking GKPositioning  GKReflexes  \
nationality age club    id                                                      
Spain       21  Arsenal 222576       15.0       8.0          11.0         9.0   
            23  Arsenal 203747       14.0      14.0          13.0        12.0   
            32  Arsenal 177604        5.0       9.0           3.0         5.0   

                               Release Clause  
nationality age club    id                     
Spain       21  Arsenal 222576          €2.5M  
            23  Arsenal 203747         €32.6M  
            32  Arsenal 177604         €15.2M  

[3 rows x 85 columns]

for the final example we will select Arselnal players from ages with the range of 22 till 25

In [70]:
#by query
df.query("age>=22 and age<=25 and club=='Arsenal'")

Unnamed: 0             Name  \
nationality        age club    id                                    
Argentina          25  Arsenal 202811        3392      E. Martínez   
Bosnia Herzegovina 25  Arsenal 207993         606     S. Kolašinac   
Egypt              25  Arsenal 213051         959        M. Elneny   
England            22  Arsenal 228295        1495       R. Holding   
                               237169       12497       C. Bramall   
Nigeria            22  Arsenal 213655         789         A. Iwobi   
Spain              23  Arsenal 203747         464  Héctor Bellerín   
Switzerland        25  Arsenal 199503         364         G. Xhaka   
Uruguay            22  Arsenal 223959         235      L. Torreira   

                                                                                Photo  \
nationality        age club    id                                                       
Argentina          25  Arsenal 202811  https://cdn.sofifa.org/players/4/19/202811.png   
Bosnia Herzegovina 25  Arsenal 207993  https://cdn.sofifa.org/players/4/19/207993.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/players/4/19/213051.png   
England            22  Arsenal 228295  https://cdn.sofifa.org/players/4/19/228295.png   
                               237169  https://cdn.sofifa.org/players/4/19/237169.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/players/4/19/213655.png   
Spain              23  Arsenal 203747  https://cdn.sofifa.org/players/4/19/203747.png   
Switzerland        25  Arsenal 199503  https://cdn.sofifa.org/players/4/19/199503.png   
Uruguay            22  Arsenal 223959  https://cdn.sofifa.org/players/4/19/223959.png   

                                                                       Flag  \
nationality        age club    id                                             
Argentina          25  Arsenal 202811   https://cdn.sofifa.org/flags/52.png   
Bosnia Herzegovina 25  Arsenal 207993    https://cdn.sofifa.org/flags/8.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/flags/111.png   
England            22  Arsenal 228295   https://cdn.sofifa.org/flags/14.png   
                               237169   https://cdn.sofifa.org/flags/14.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/flags/133.png   
Spain              23  Arsenal 203747   https://cdn.sofifa.org/flags/45.png   
Switzerland        25  Arsenal 199503   https://cdn.sofifa.org/flags/47.png   
Uruguay            22  Arsenal 223959   https://cdn.sofifa.org/flags/60.png   

                                       Overall  Potential  \
nationality        age club    id                           
Argentina          25  Arsenal 202811       72         77   
Bosnia Herzegovina 25  Arsenal 207993       79         82   
Egypt              25  Arsenal 213051       77         79   
England            22  Arsenal 228295       76         83   
                               237169       63         70   
Nigeria            22  Arsenal 213655       78         84   
Spain              23  Arsenal 203747       80         85   
Switzerland        25  Arsenal 199503       81         84   
Uruguay            22  Arsenal 223959       82         87   

                                                                        Club Logo  \
nationality        age club    id                                                   
Argentina          25  Arsenal 202811  https://cdn.sofifa.org/teams/2/light/1.png   
Bosnia Herzegovina 25  Arsenal 207993  https://cdn.sofifa.org/teams/2/light/1.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/teams/2/light/1.png   
England            22  Arsenal 228295  https://cdn.sofifa.org/teams/2/light/1.png   
                               237169  https://cdn.sofifa.org/teams/2/light/1.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/teams/2/light/1.png   
Spain              23  Arsenal 203747  https://cdn.sofifa.org/team

In [73]:
#by loc
df.loc[(slice(None),slice(22,25),"Arsenal"),:]

Unnamed: 0             Name  \
nationality        age club    id                                    
Argentina          25  Arsenal 202811        3392      E. Martínez   
Bosnia Herzegovina 25  Arsenal 207993         606     S. Kolašinac   
Egypt              25  Arsenal 213051         959        M. Elneny   
England            22  Arsenal 228295        1495       R. Holding   
                               237169       12497       C. Bramall   
Nigeria            22  Arsenal 213655         789         A. Iwobi   
Spain              23  Arsenal 203747         464  Héctor Bellerín   
Switzerland        25  Arsenal 199503         364         G. Xhaka   
Uruguay            22  Arsenal 223959         235      L. Torreira   

                                                                                Photo  \
nationality        age club    id                                                       
Argentina          25  Arsenal 202811  https://cdn.sofifa.org/players/4/19/202811.png   
Bosnia Herzegovina 25  Arsenal 207993  https://cdn.sofifa.org/players/4/19/207993.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/players/4/19/213051.png   
England            22  Arsenal 228295  https://cdn.sofifa.org/players/4/19/228295.png   
                               237169  https://cdn.sofifa.org/players/4/19/237169.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/players/4/19/213655.png   
Spain              23  Arsenal 203747  https://cdn.sofifa.org/players/4/19/203747.png   
Switzerland        25  Arsenal 199503  https://cdn.sofifa.org/players/4/19/199503.png   
Uruguay            22  Arsenal 223959  https://cdn.sofifa.org/players/4/19/223959.png   

                                                                       Flag  \
nationality        age club    id                                             
Argentina          25  Arsenal 202811   https://cdn.sofifa.org/flags/52.png   
Bosnia Herzegovina 25  Arsenal 207993    https://cdn.sofifa.org/flags/8.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/flags/111.png   
England            22  Arsenal 228295   https://cdn.sofifa.org/flags/14.png   
                               237169   https://cdn.sofifa.org/flags/14.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/flags/133.png   
Spain              23  Arsenal 203747   https://cdn.sofifa.org/flags/45.png   
Switzerland        25  Arsenal 199503   https://cdn.sofifa.org/flags/47.png   
Uruguay            22  Arsenal 223959   https://cdn.sofifa.org/flags/60.png   

                                       Overall  Potential  \
nationality        age club    id                           
Argentina          25  Arsenal 202811       72         77   
Bosnia Herzegovina 25  Arsenal 207993       79         82   
Egypt              25  Arsenal 213051       77         79   
England            22  Arsenal 228295       76         83   
                               237169       63         70   
Nigeria            22  Arsenal 213655       78         84   
Spain              23  Arsenal 203747       80         85   
Switzerland        25  Arsenal 199503       81         84   
Uruguay            22  Arsenal 223959       82         87   

                                                                        Club Logo  \
nationality        age club    id                                                   
Argentina          25  Arsenal 202811  https://cdn.sofifa.org/teams/2/light/1.png   
Bosnia Herzegovina 25  Arsenal 207993  https://cdn.sofifa.org/teams/2/light/1.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/teams/2/light/1.png   
England            22  Arsenal 228295  https://cdn.sofifa.org/teams/2/light/1.png   
                               237169  https://cdn.sofifa.org/teams/2/light/1.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/teams/2/light/1.png   
Spain              23  Arsenal 203747  https://cdn.sofifa.org/team

we can do better
for the use of loc you need to remember the order of your index and always add slice(None) to the indexes you want all of.

and using query is a bit vebuse and slower the the use of loc


with fast index

In [75]:
df.fidx.slice(nationality="Spain")

Unnamed: 0             Name  \
nationality age club                    id                                    
Spain       16  CF Rayo Majadahonda     246465       12828         Guerrero   
                Real Sporting de Gijón  245616       12496   Pelayo Morilla   
            17  Atlético Madrid         245999       15786          Mollejo   
                Borussia Dortmund       241811        7217     Sergio Gómez   
                Elche CF                243943       16224       Javi López   
                Manchester City         245037       13211      Eric García   
                Málaga CF               246070       15422       Iván Jaime   
                RCD Mallorca            246455       15104      Pablo Ramón   
                Rayo Vallecano          245632       16467            Morro   
                                        246278       14662     Manu Navarro   
                Real Betis              245894       14321           Robert   
                SD Huesca               246593       16756           Javito   
            18  Albacete BP             243156       15168    Álvaro García   
                                        245599       16792      Darío Ramos   
                Athletic Club de Bilbao 239614       12059      Jesús Areso   
                                        244675        8502           Sancet   
                Atlético Madrid         240559       15645            Obama   
                                        243652       11805     Borja Garcés   
                CF Reus Deportiu        245705       15697      Iker Goujón   
                Córdoba CF              246044       13157    Andrés Martín   
                FC Barcelona            237522       10777     Jorge Cuenca   
                                        239250        8289        Abel Ruiz   
                                        240915        8857          Miranda   
                                        242816        6102       Riqui Puig   
                Getafe CF               243032       13367        Hugo Duro   
                Girona FC               243576        6076      Pedro Porro   
                                        246351       15967           Valery   
                Manchester City         231410        6559      Brahim Díaz   
                RC Celta                241055       14393     Diego Pampín   
                RCD Espanyol            244407       16556       Pol Lozano   
...                                                    ...              ...   
            35  AD Alcorcón             145117        4627            Dorca   
                CD Lugo                 145996        2224           Seoane   
                CF Reus Deportiu        171565        5807          Linares   
                Extremadura UD          171017        8805      Íñigo López   
                Milan                   24630          224       Pepe Reina   
                Perth Glory             147006        1692     Diego Castro   
                Queens Park Rangers     153938        3464     Àngel Rangel   
                RCD Espanyol            135697         911    Sergio García   
                Rayo Vallecano          110019        4551           Dorado   
                Real Oviedo             157270        5941            Toché   
                UD Las Palmas           146058        5238             Momo   
            36  Getafe CF               154472        1019     Jorge Molina   
                New York City FC        113422         315      David Villa   
                RCD Espanyol            146748         544      Diego López   
                RCD Mallorca            110680        9179     Xisco Campos   
                Rayo Vallecano          139597        5039      Javi Guerra   
                Real Betis              45186          411          Joaquín   
                UD Las Palmas           146059        3644     David García   
                Ven

In [78]:
df.fidx.slice(age=25)

Unnamed: 0               Name  \
nationality age club                   id                                      
Albania     25  Atalanta               208165        7815        B. Djimsiti   
                Empoli                 203780        4767          F. Veseli   
                Neuchâtel Xamax        244581       14912      A. Fejzullahu   
Algeria     25  Dijon FCO              188600        3770           M. Abeid   
                FC Metz                210438        3232         F. Boulaya   
                FC Sion                243896       15665      A. Abdellaoui   
                Le Havre AC            222515        3371          Z. Ferhat   
Angola      25  Club Brugge KV         208364        3902       Clinton Mata   
                Livingston FC          207651       12207        Dolly Menga   
                Zagłębie Sosnowiec     209842       14447         Christovao   
Argentina   25  Ajax                   211256         445      N. Tagliafico   
                Argentinos Juniors     214776        5612         O. Benítez   
                                       226675       11460     F. Lanzillotta   
                Arsenal                202811        3392        E. Martínez   
                Atalanta               215259         784          E. Rigoni   
                Atlético Tucumán       227602        5706         F. Álvarez   
                Bayer 04 Leverkusen    215353         591          L. Alario   
                Belgrano de Córdoba    215328        6728      G. Gil Romero   
                CD Antofagasta         219518        9330          P. Becker   
                Club Atlético Banfield 222018       11280         R. Arciero   
                Club Atlético Colón    215070        2119           G. Ortíz   
                                       223593        3536          J. Correa   
                Club Atlético Huracán  215168        3615           J. Garro   
                                       219931        7766         D. Mendoza   
                                       242080       12693          G. Guerra   
                Club Atlético Lanús    202331        3492         F. Quignon   
                                       214779        8752           B. Vides   
                Club Atlético Talleres 214665        2931         J. Ramírez   
                Club Deportes Temuco   227876       12214         L. Campana   
                Clube Sport Marítimo   220938        3245          J. Correa   
...                                                   ...                ...   
Uruguay     25  Independiente Santa Fe 205972       12643     R. Bentancourt   
                Jaguares de Córdoba    224078       15471          F. Pintos   
                KV Kortrijk            219988        4236        F. Avenatti   
                Milan                  213400         790          D. Laxalt   
                Millonarios FC         239645        5749   M. de los Santos   
                Palermo                220084        3759        C. Falletti   
                Patronato              235950       13663         F. Barceló   
                Pescara                205726        3698         G. Brugman   
                San Martín de San Juan 244954       13813   A. Silva Quiroga   
                Sivasspor              223690        6390          S. Rochet   
                Unión Española         239104        7648        A. Pizzorno   
                Unión La Calera        227838        6606           C. Núñez   
                                       242209        7797       A. Rodriguez   
                Vancouver Whitecaps FC 235280        7715             J. Aja   
Venezuela   25  Al Wehda               217097        1590           R. Otero   
                Atlanta United         207877         345        J. Martínez   
                CF Reus Deportiu       233329        7076      M. Villanueva   
                Elche CF               207876  

In [79]:
df.fidx.slice(nationality="Spain",club="Arsenal")

Unnamed: 0             Name  \
nationality age club    id                                    
Spain       21  Arsenal 222576        7069      Pleguezuelo   
            23  Arsenal 203747         464  Héctor Bellerín   
            32  Arsenal 177604         520    Nacho Monreal   

                                                                         Photo  \
nationality age club    id                                                       
Spain       21  Arsenal 222576  https://cdn.sofifa.org/players/4/19/222576.png   
            23  Arsenal 203747  https://cdn.sofifa.org/players/4/19/203747.png   
            32  Arsenal 177604  https://cdn.sofifa.org/players/4/19/177604.png   

                                                               Flag  Overall  \
nationality age club    id                                                     
Spain       21  Arsenal 222576  https://cdn.sofifa.org/flags/45.png       68   
            23  Arsenal 203747  https://cdn.sofifa.org/flags/45.png       80   
            32  Arsenal 177604  https://cdn.sofifa.org/flags/45.png       80   

                                Potential  \
nationality age club    id                  
Spain       21  Arsenal 222576         77   
            23  Arsenal 203747         85   
            32  Arsenal 177604         80   

                                                                 Club Logo  \
nationality age club    id                                                   
Spain       21  Arsenal 222576  https://cdn.sofifa.org/teams/2/light/1.png   
            23  Arsenal 203747  https://cdn.sofifa.org/teams/2/light/1.png   
            32  Arsenal 177604  https://cdn.sofifa.org/teams/2/light/1.png   

                                 Value  Wage  Special  ... Composure  Marking  \
nationality age club    id                             ...                      
Spain       21  Arsenal 222576   €1.2M  €21K     1672  ...      59.0     65.0   
            23  Arsenal 203747  €16.5M  €85K     1990  ...      70.0     75.0   
            32  Arsenal 177604     €8M  €99K     2011  ...      76.0     80.0   

                                StandingTackle  SlidingTackle GKDiving  \
nationality age club    id                                               
Spain       21  Arsenal 222576            68.0           70.0     14.0   
            23  Arsenal 203747            78.0           80.0      8.0   
            32  Arsenal 177604            83.0           81.0      7.0   

                               GKHandling GKKicking GKPositioning  GKReflexes  \
nationality age club    id                                                      
Spain       21  Arsenal 222576       15.0       8.0          11.0         9.0   
            23  Arsenal 203747       14.0      14.0          13.0        12.0   
            32  Arsenal 177604        5.0       9.0           3.0         5.0   

                               Release Clause  
nationality age club    id                     
Spain       21  Arsenal 222576          €2.5M  
            23  Arsenal 203747         €32.6M  
            32  Arsenal 177604         €15.2M  

[3 rows x 85 columns]

In [80]:
df.fidx.slice(age=slice(22,25),club="Arsenal")

Unnamed: 0             Name  \
nationality        age club    id                                    
Argentina          25  Arsenal 202811        3392      E. Martínez   
Bosnia Herzegovina 25  Arsenal 207993         606     S. Kolašinac   
Egypt              25  Arsenal 213051         959        M. Elneny   
England            22  Arsenal 228295        1495       R. Holding   
                               237169       12497       C. Bramall   
Nigeria            22  Arsenal 213655         789         A. Iwobi   
Spain              23  Arsenal 203747         464  Héctor Bellerín   
Switzerland        25  Arsenal 199503         364         G. Xhaka   
Uruguay            22  Arsenal 223959         235      L. Torreira   

                                                                                Photo  \
nationality        age club    id                                                       
Argentina          25  Arsenal 202811  https://cdn.sofifa.org/players/4/19/202811.png   
Bosnia Herzegovina 25  Arsenal 207993  https://cdn.sofifa.org/players/4/19/207993.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/players/4/19/213051.png   
England            22  Arsenal 228295  https://cdn.sofifa.org/players/4/19/228295.png   
                               237169  https://cdn.sofifa.org/players/4/19/237169.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/players/4/19/213655.png   
Spain              23  Arsenal 203747  https://cdn.sofifa.org/players/4/19/203747.png   
Switzerland        25  Arsenal 199503  https://cdn.sofifa.org/players/4/19/199503.png   
Uruguay            22  Arsenal 223959  https://cdn.sofifa.org/players/4/19/223959.png   

                                                                       Flag  \
nationality        age club    id                                             
Argentina          25  Arsenal 202811   https://cdn.sofifa.org/flags/52.png   
Bosnia Herzegovina 25  Arsenal 207993    https://cdn.sofifa.org/flags/8.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/flags/111.png   
England            22  Arsenal 228295   https://cdn.sofifa.org/flags/14.png   
                               237169   https://cdn.sofifa.org/flags/14.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/flags/133.png   
Spain              23  Arsenal 203747   https://cdn.sofifa.org/flags/45.png   
Switzerland        25  Arsenal 199503   https://cdn.sofifa.org/flags/47.png   
Uruguay            22  Arsenal 223959   https://cdn.sofifa.org/flags/60.png   

                                       Overall  Potential  \
nationality        age club    id                           
Argentina          25  Arsenal 202811       72         77   
Bosnia Herzegovina 25  Arsenal 207993       79         82   
Egypt              25  Arsenal 213051       77         79   
England            22  Arsenal 228295       76         83   
                               237169       63         70   
Nigeria            22  Arsenal 213655       78         84   
Spain              23  Arsenal 203747       80         85   
Switzerland        25  Arsenal 199503       81         84   
Uruguay            22  Arsenal 223959       82         87   

                                                                        Club Logo  \
nationality        age club    id                                                   
Argentina          25  Arsenal 202811  https://cdn.sofifa.org/teams/2/light/1.png   
Bosnia Herzegovina 25  Arsenal 207993  https://cdn.sofifa.org/teams/2/light/1.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/teams/2/light/1.png   
England            22  Arsenal 228295  https://cdn.sofifa.org/teams/2/light/1.png   
                               237169  https://cdn.sofifa.org/teams/2/light/1.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/teams/2/light/1.png   
Spain              23  Arsenal 203747  https://cdn.sofifa.org/team

or

In [81]:
df.fidx.slice(age=pd.IndexSlice[22:25],club="Arsenal")

Unnamed: 0             Name  \
nationality        age club    id                                    
Argentina          25  Arsenal 202811        3392      E. Martínez   
Bosnia Herzegovina 25  Arsenal 207993         606     S. Kolašinac   
Egypt              25  Arsenal 213051         959        M. Elneny   
England            22  Arsenal 228295        1495       R. Holding   
                               237169       12497       C. Bramall   
Nigeria            22  Arsenal 213655         789         A. Iwobi   
Spain              23  Arsenal 203747         464  Héctor Bellerín   
Switzerland        25  Arsenal 199503         364         G. Xhaka   
Uruguay            22  Arsenal 223959         235      L. Torreira   

                                                                                Photo  \
nationality        age club    id                                                       
Argentina          25  Arsenal 202811  https://cdn.sofifa.org/players/4/19/202811.png   
Bosnia Herzegovina 25  Arsenal 207993  https://cdn.sofifa.org/players/4/19/207993.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/players/4/19/213051.png   
England            22  Arsenal 228295  https://cdn.sofifa.org/players/4/19/228295.png   
                               237169  https://cdn.sofifa.org/players/4/19/237169.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/players/4/19/213655.png   
Spain              23  Arsenal 203747  https://cdn.sofifa.org/players/4/19/203747.png   
Switzerland        25  Arsenal 199503  https://cdn.sofifa.org/players/4/19/199503.png   
Uruguay            22  Arsenal 223959  https://cdn.sofifa.org/players/4/19/223959.png   

                                                                       Flag  \
nationality        age club    id                                             
Argentina          25  Arsenal 202811   https://cdn.sofifa.org/flags/52.png   
Bosnia Herzegovina 25  Arsenal 207993    https://cdn.sofifa.org/flags/8.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/flags/111.png   
England            22  Arsenal 228295   https://cdn.sofifa.org/flags/14.png   
                               237169   https://cdn.sofifa.org/flags/14.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/flags/133.png   
Spain              23  Arsenal 203747   https://cdn.sofifa.org/flags/45.png   
Switzerland        25  Arsenal 199503   https://cdn.sofifa.org/flags/47.png   
Uruguay            22  Arsenal 223959   https://cdn.sofifa.org/flags/60.png   

                                       Overall  Potential  \
nationality        age club    id                           
Argentina          25  Arsenal 202811       72         77   
Bosnia Herzegovina 25  Arsenal 207993       79         82   
Egypt              25  Arsenal 213051       77         79   
England            22  Arsenal 228295       76         83   
                               237169       63         70   
Nigeria            22  Arsenal 213655       78         84   
Spain              23  Arsenal 203747       80         85   
Switzerland        25  Arsenal 199503       81         84   
Uruguay            22  Arsenal 223959       82         87   

                                                                        Club Logo  \
nationality        age club    id                                                   
Argentina          25  Arsenal 202811  https://cdn.sofifa.org/teams/2/light/1.png   
Bosnia Herzegovina 25  Arsenal 207993  https://cdn.sofifa.org/teams/2/light/1.png   
Egypt              25  Arsenal 213051  https://cdn.sofifa.org/teams/2/light/1.png   
England            22  Arsenal 228295  https://cdn.sofifa.org/teams/2/light/1.png   
                               237169  https://cdn.sofifa.org/teams/2/light/1.png   
Nigeria            22  Arsenal 213655  https://cdn.sofifa.org/teams/2/light/1.png   
Spain              23  Arsenal 203747  https://cdn.sofifa.org/team

you can still use by order

In [82]:
df.fidx.slice("Spain",23)

Unnamed: 0  \
nationality age club                    id                   
Spain       23  1. FC Nürnberg          223095       10406   
                Albacete BP             209695        6723   
                Arsenal                 203747         464   
                Athletic Club de Bilbao 227127         956   
                                        227950        1026   
                Atlético Madrid         208421          83   
                CA Osasuna              212261        3334   
                                        238305        4643   
                CD Leganés              214944        4459   
                                        224894       14293   
                                        231530        5143   
                                        238882        7803   
                CD Lugo                 241283       14308   
                CD Numancia             211428        7528   
                                        240722       14171   
                                        243972       13078   
                CD Tenerife             215568        3268   
                                        221452        4806   
                                        229361        6552   
                                        235088        5924   
                                        235518       13059   
                                        242201        2663   
                CF Rayo Majadahonda     214929        8283   
                                        219637       13027   
                                        223056       10266   
                                        243558       10353   
                CF Reus Deportiu        242397       15202   
                                        243976        7679   
                Chelsea                 206585         177   
                Cádiz CF                220190       17043   
...                                                    ...   
                Levante UD              213063        2208   
                                        215616        1702   
                Milan                   210617         246   
                Málaga CF               227106        4070   
                RC Celta                219848        5448   
                RCD Espanyol            229668        1252   
                RCD Mallorca            210747       10176   
                                        225121        8086   
                Rayo Vallecano          210315        1409   
                Real Betis              221087         237   
                Real Madrid             226161         573   
                Real Oviedo             226157        6029   
                Real Sociedad           225562        6701   
                                        238126       10126   
                                        245520        4017   
                Real Sporting de Gijón  235623        2279   
                                        236635        9191   
                                        236997        9662   
                Real Valladolid CF      229517        2944   
                Real Zaragoza           211857        7205   
                                        239010        7282   
                SD Eibar                212488       13091   
                                        224309         757   
                Sint-Truidense VV       211323        8186   
                UD Almería              228309       12916   
                                        235093        9160   
                                        245550       12256   
                UD Las Palmas           245528       10035   
                Valencia CF             211688         443   
                Villarreal CF           224164       14413   

                                                                Name  \
nationality age club                    id                           

In [84]:
df.fidx.slice("Spain",slice(22,26),"Arsenal")

,,,,Unnamed: 0,Name,Photo,Flag,Overall,Potential,Club Logo,Value,Wage,Special,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
nationality,age,club,id,,,,,,,,,,,,,,,,,,,,,
Spain,23,Arsenal,203747,464,Héctor Bellerín,https://cdn.sofifa.org/players/4/19/203747.png,https://cdn.sofifa.org/flags/45.png,80,85,https://cdn.sofifa.org/teams/2/light/1.png,€16.5M,€85K,1990,...,70.0,75.0,78.0,80.0,8.0,14.0,14.0,13.0,12.0,€32.6M


you can get the slice for setting values

In [88]:
slc = df.fidx.f_slice("Spain",slice(22,26),"Arsenal")

In [89]:
df.loc[slc,"Overall"] = 3

In [90]:
df.loc[slc,"Overall"] 

nationality  age  club     id    
Spain        23   Arsenal  203747    3
Name: Overall, dtype: int64